In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

In [ ]:
train = pd.read_csv('Data/train.csv')

In [ ]:
fraud = train['isFraud']
X_train = train.drop('isFraud',axis=1)
del train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, fraud, test_size = 0.5, shuffle=False)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
model = xgb.XGBClassifier(
    max_depth = 300,
    learning_rate = 0.1,
    n_estimators = 300,
    objective = 'binary:logistic'       
)

train_rows = 100000

eval_set = [(X_train.iloc[train_rows:,:],y_train.iloc[train_rows:])]

model.fit(X_train.iloc[:train_rows,:], y_train.iloc[:train_rows], 
          eval_metric = "auc", 
          eval_set= eval_set,
          early_stopping_rounds = 5
         )

In [ ]:
preds = model.predict_proba(X_test)

score = roc_auc_score(y_test, preds[:,1])

In [ ]:
print(score)

In [ ]:
model.evals_result()['validation_0']['auc'][-1]

In [ ]:
model = xgb.XGBClassifier(
    max_depth = 300,
    learning_rate = 0.1,
    n_estimators = 100,
    objective = 'binary:logistic'
       
)

rounds = np.arange(1,7)
test_scores = []
for round in rounds:
    model.fit(X_train.iloc[:train_rows,:], y_train.iloc[:train_rows], 
          eval_metric = "auc", 
          eval_set= eval_set,
          early_stopping_rounds = round
         )
    last_val_score = model.evals_result()['validation_0']['auc'][-1]
    
    preds = model.predict_proba(X_test)
    test_score = roc_auc_score(y_test, preds[:,1])
    print(test_score)
    
    scores = [last_val_score, test_score]
    test_scores.append(scores)

In [ ]:
print(test_scores)